# Protocol for a distributed synthetic market

This is a protocol for matching **state contingent contracts** linear between an upper and lower bound and at a particular expiry.

<img src="img/BFC.png"  width=500>

These contracts can be combined to construct standard forward and option contracts or event markets (payoffs depending on a discrete). 

<img src="img/StackedBFC.png" width=500>

The protocol uses a chain of signatures to verify orders. Orders are accepted by the market if they have a valid signature and the trader's positions satisfiy a collateral check across all possible worst outcomes.

Markets are settled by the market owner adding a new upper and lower bound for a market equal to a single number. For example, the lower bound for each market is the highest lower bound for itself of any sub-market or any previous.

[figure]


## Data structures

### A market

A **market** is defined by:

- 'marketRootId': (integer) market id
- 'marketBranchId': (integer) sub-markets >1 (sub-markets bounded by super-markets)
- 'marketMin' - (float) minimum possible outcome
- 'marketMax' - (float) maximum possible outcome
- 'traderId' - (integer) market owner trader ID
- 'signatureMsg' - (string) message for signature
- 'signature' - (binary) signed message

E.g.

Market with (root = 3, branch = 1); market bounded between (0, 1); owned by trader 1 and signed.

~~~~

testMarket = struct('marketRootId', 3, 'marketBranchId', 1,...
                                'marketMin', 0, 'marketMax', 1,...
                                'traderId', 1, 'signatureMsg','sigmsg1',...
                                'signature', 'sig1')

~~~~

- Sub markets have marketMin/marketMax bounded by superMarkets.
- Any amount of valid markets with the same root/branch can be added and the market will take the highest minimum and the lowest maximum.


### An order

An **order** in a market is defined by:

 - 'tradeRootId': (integer) trade root id
 - 'tradeBranchId': (integer) subtrades (dependent trades such as offsets and partials)
 - 'isMatched': (boolean) is trade matched (adding an unmatched trade requries adding an offsetting trade and adding an equal sized matched trade from cache)
 - 'price':  (float) price of trade 
 - 'quantity':  (float) quanitity of trade (positive quantity for bids, negative for offers) 
 - 'marketRootId' : (integer) market id
 - 'marketBranchId': (integer) sub-markets > 1 (sub-markets bounded by super-markets)
 - 'traderId':  (integer) trade owner trader ID
 - 'tradeType': (integer) Primary = 1, pArtial = 1, Offset = 3, Reduction = 4
 - 'signatureMsg' - (string) message for signature
 - 'signature' - (binary) signed message

E.g. 

Unmatched primary trade (price = 0.5, quantity = 10) on market (root = 1, branch = 1)

~~~~

testTrade = struct('traderId', 1, 'tradeRootId', 1, 'tradeBranchId', 1,...
                'isMatched', 0, 'price', 0.5,...
                'quantity', 10, 'marketRootId', 3,...
                'marketBranchId', 1,'signatureMsg',...
                'tradeType, 1,'sigMsg', 'signature', 'sig');

~~~~


[Dictionary trade example]

Trades can be added to the order book but not removed or changed except for the 'tradeType' field which determines if the trade should be considered in collateral calculations.

#### Geometry of an order

Since orders can only be added to the order book and not subtracted, creating a matched order requires an offset of the unmatched order and a new matched order of equal size.

Consider an order '(p=0.5, q=0.5)'. For the trade to successfully be matched it requires at minimum:

- Primary  (p=0.5, q=10, isMatched = 0) 
- Offset  (p=0.5, q=-10, isMatched = 0)
- Match (p=0.5, q=10, isMatchced = 1)

Optionally, the trade an also include a reduction trade for the market to use to kill an existing order to make way for a new order. These don't need offsets.

- Reduction  (p=0.5, q=10*-0.1=-1, isMatched=1)

Geometrically, this set of trades can be represented in (p,q) space:

<img src="img\OrderGeometry1.png" width = 600>

It is useful to allow partial fills (lower quantity and weakly better price). Each partial trade only requires a matched version and not an offset.

[Geometry of single trade with partial fill region]

Any non-primary trade that is picked up for a match is set as primary by the system so it can be picked up in future collateral calculations (this is the only changable field in the order book).


#### Geometry of signature chain 

Each trade is chained to a previous trade according to some rule. The order book will require a valid signature (valid signature for message and valid previous node) and a collateral check. Primary and partial orders are chained to a previous valid order, while offset and reductions are chained to the primary trade.

[Pictures: 
Previous valid -> Primary -> Primary offset cone -> Primary matched rectangle
Previous valid -> Primary -> Partial -> Partial offset cone -> Matched rectangle
]

## Methods


[methods]


1+2




$x^2/2$

$\int_{-\infty}^{\infty} f(x) dx$

$poop^{girlfriend}$

In [0]:
Here's our logo (hover to see the title text):

Inline-style: 
![alt text](https://github.com/adam-p/markdown-here/raw/master/sr